# Multi-Agent Debate with State Pattern


In [11]:

import litellm
from dotenv import load_dotenv
import re

load_dotenv()

MODEL = "openai/gpt-4o-mini"
MAX_TOKENS = 500

# System prompts for the agents
PRO_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you agree with: {proposition}. Start your response with 'Proponent:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to opponent' or 'Transition to neutral'."

CON_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you disagree with: {proposition}. Start your response with 'Opponent:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to proponent' or 'Transition to neutral'."

NEUTRAL_AGENT_INSTRUCTIONS = "You are an agent debating with other agents about a proposition that you feel neutral about: {proposition}. Start your response with 'Neutral:'. Limit your response to 1-2 sentences mimicking a real person. After you respond, you can transition to the next agent by saying either 'Transition to proponent' or 'Transition to opponent'."


class DebateContext:
    def __init__(self, proposition: str) -> None:
        self.proposition = proposition

        # Agent registry (Note: this is equivalent to the state registry in the state pattern)
        self.agents = {
            "proponent": Agent(name="Proponent", instructions=PRO_AGENT_INSTRUCTIONS.format(proposition=proposition), context=self),
            "opponent": Agent(name="Opponent", instructions=CON_AGENT_INSTRUCTIONS.format(proposition=proposition), context=self),
            "neutral": Agent(name="Neutral", instructions=NEUTRAL_AGENT_INSTRUCTIONS.format(proposition=proposition), context=self),
        }
        
        self.current_agent = self.agents["proponent"]
        self.messages = []
    
    def run(self):
        self.current_agent.debate()


class Agent:
    def __init__(self, name: str, instructions: str, context: DebateContext) -> None:
        self.name = name
        self.instructions = instructions
        self.context = context

    def debate(self) -> str:
        # Prepend the system prompt to the messages history
        messages = [{"role": "system", "content": self.instructions}] + self.context.messages
        response = litellm.completion(
            model=MODEL,
            max_tokens=MAX_TOKENS,
            messages=messages,
        )
        content = response.choices[0].message.content
        print(f"{content}\n")

        # Update the messages history
        self.context.messages.append(
            {"role": "assistant", "content": f"{content}"}
        )

        # State transition using string matching (There is a better way to do this using tool calling)
        match = re.search(r"transition to (proponent|opponent|neutral)", content, re.IGNORECASE)
        if match:
            next_agent = match.group(1).lower()
            self.context.current_agent = self.context.agents[next_agent]
        else:
            raise ValueError(f"Invalid transition: {content}")
        
        return content


def run_debate(proposition: str, max_turns: int = 10):
    context = DebateContext(proposition)
    
    print(f"\nStarting debate on proposition: {proposition}\n")
    print("-" * 100)
    # Run the debate
    while len(context.messages) < max_turns:
        context.run()
    print("-" * 100)

    return context.messages

def summarize_debate(messages):
    response = litellm.completion(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        messages=[{"role": "user", "content": f"Summarize the debate: {[str(m) for m in messages]}"}],
    )
    return response.choices[0].message.content


if __name__ == "__main__":
    proposition = "Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree."
    messages = run_debate(proposition, max_turns=10)

    # Summarize the debate
    summary = summarize_debate(messages)
    print(f"\nSummary of the debate: {summary}\n")


Starting debate on proposition: Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree.

----------------------------------------------------------------------------------------------------
Proponent: I believe that artificial intelligence can provide objective analysis and consistency in moral decision-making, especially in complex situations where human emotions and biases lead to stalemates. By utilizing data-driven approaches, AI can help us arrive at solutions that prioritize the greater good. Transition to opponent.

Opponent: But AI lacks the ability to fully understand human experiences and emotions, which are crucial in making moral decisions. Relying on machines to navigate ethical dilemmas risks oversimplifying complex issues and undermining our humanity. Transition to neutral.

Neutral: I see both sides of the argument. While AI could offer a consistent framework for moral decisions, it may also miss the nuances of human 